In [ ]:
from google.colab import files
import pandas as pd
from openpyxl import load_workbook
from openpyxl.styles import PatternFill, Alignment, Font
from datetime import datetime, timedelta

def format_value(value):
    if isinstance(value, (int, float)):
        return f"R$ {value:,.2f}".replace(',', 'v').replace('.', ',').replace('v', '.')
    else:
        value = value.replace("'", ".")
        return f"R$ {float(value):,.2f}".replace(',', 'v').replace('.', ',').replace('v', '.')

uploaded = files.upload()

sheet_names = ['AVARIAS', 'Avarias', 'avarias', 'Sublocação']
columns_to_extract_avarias = ['Motivo da O.S', 'TOTAL REEMBOLSAVEL II', 'placa']
columns_to_extract_sublocacao = ['Placa Sublocado', 'TOTAL REEMBOLSAVEL II', 'Motivo do Chamado', 'Descrição']

combined_data = pd.DataFrame()

for filename in uploaded.keys():
    if filename.endswith('.xlsx') or filename.endswith('.xls'):
        for sheet in sheet_names:
            try:
                df = pd.read_excel(filename, sheet_name=sheet)
                print(f'Aba carregada: {sheet} do arquivo {filename}')

                if sheet in ['AVARIAS', 'Avarias', 'avarias']:
                    if df['Motivo da O.S'].isnull().all() or (df['Motivo da O.S'] == '').all():
                        df = df.iloc[:-1]  # Remove a última linha

                    extracted_data = df[columns_to_extract_avarias].copy()
                    extracted_data.rename(columns={'Motivo da O.S': 'Item', 'TOTAL REEMBOLSAVEL II': 'Valor total', 'placa': 'Veículo'}, inplace=True)
                    extracted_data['Item'] = extracted_data['Item'].replace('CORRETIVA', 'Manutenção/Corretiva')
                    extracted_data['Data/hora'] = ''
                    extracted_data['Colaborador'] = ''
                    extracted_data['Fornecedor'] = 'MOVIDA'
                    extracted_data['n°documento'] = ''
                    extracted_data['Quantidade'] = '1'
                    extracted_data['Valor Unitário'] = ''
                    extracted_data['Valor'] = ''
                    extracted_data['Hodometro'] = ''
                    extracted_data['Garantia Dias'] = ''
                    extracted_data['Garantia ultilizacao'] = ''
                    extracted_data['Desconto Reais'] = ''
                    extracted_data['Desconto %'] = ''
                    extracted_data['Valor total'] = extracted_data['Valor total'].apply(format_value)
                    combined_data = pd.concat([combined_data, extracted_data], ignore_index=True)

                elif sheet == 'Sublocação':
                    sublocacao_extracted = df[columns_to_extract_sublocacao].copy()
                    sublocacao_extracted.rename(columns={'Placa Sublocado': 'Veículo', 'TOTAL REEMBOLSAVEL II': 'Valor total', 'Motivo do Chamado': 'Item', 'Descrição': 'Observação'}, inplace=True)
                    sublocacao_extracted['Fornecedor'] = 'MOVIDA'
                    sublocacao_extracted['Item'] = sublocacao_extracted['Item'].replace('CORRETIVA', 'Manutenção/Corretiva')
                    sublocacao_extracted['Quantidade'] = '1'
                    sublocacao_extracted['Valor total'] = sublocacao_extracted['Valor total'].apply(format_value)
                    combined_data = pd.concat([combined_data, sublocacao_extracted], ignore_index=True)

            except ValueError:
                print(f'Aba "{sheet}" não encontrada no arquivo {filename}.')

date_input = input("Digite a data e hora (dd/mm/aaaa HH:MM): ")
initial_datetime = datetime.strptime(date_input, "%d/%m/%Y %H:%M")

combined_data['Data/hora'] = [
    (initial_datetime + timedelta(minutes=i)).strftime("%d/%m/%Y %H:%M:%S")
    for i in range(len(combined_data))
]


new_order = ['Data/hora', 'Colaborador', 'Fornecedor', 'n°documento', 'Veículo', 'Item', 'Quantidade', 'Valor Unitário', 'Valor', 'Valor total', 'Hodometro', 'Garantia Dias', 'Garantia ultilizacao', 'Desconto Reais', 'Desconto %', 'Observação']
combined_data = combined_data.reindex(columns=new_order)

new_filename = 'dados_combinados.xlsx'
combined_data.to_excel(new_filename, index=False)
print(f'Dados combinados salvos em: {new_filename}')

workbook = load_workbook(new_filename)
sheet = workbook.active

columns_to_paint = ['Data/hora', 'Fornecedor', 'Veículo', 'Item', 'Quantidade', 'Valor total', 'Observação']


for col in range(1, sheet.max_column + 1):
    cell = sheet.cell(row=1, column=col)
    cell.alignment = Alignment(horizontal='left')
    cell.font = Font(bold=False)


    if combined_data.columns[col - 1] in columns_to_paint:
        cell.fill = PatternFill(start_color='FFFF00', end_color='FFFF00', fill_type='solid')

for column in sheet.columns:
    max_length = 0
    column_letter = column[0].column_letter
    for cell in column:
        try:
            current_length = len(str(cell.value))
            if current_length > max_length:
                max_length = current_length
        except:
            pass
    adjusted_width = (max_length + 2)
    sheet.column_dimensions[column_letter].width = adjusted_width

workbook.save(new_filename)

files.download(new_filename)


Saving BRK - PORTO FERREIRA - 09-2024 (1).xlsx to BRK - PORTO FERREIRA - 09-2024 (1).xlsx
Saving BRK - RIO CLARO - 09-2024.xlsx to BRK - RIO CLARO - 09-2024.xlsx
Saving BRK - SANTA GERTRUDES - 09-2024.xlsx to BRK - SANTA GERTRUDES - 09-2024.xlsx
Saving BRK - LIMEIRA - 09-2024.xlsx to BRK - LIMEIRA - 09-2024.xlsx
Saving BRK - SUMARE - 09-2024 (2).xlsx to BRK - SUMARE - 09-2024 (2).xlsx
Aba "AVARIAS" não encontrada no arquivo BRK - PORTO FERREIRA - 09-2024 (1).xlsx.
Aba carregada: Avarias do arquivo BRK - PORTO FERREIRA - 09-2024 (1).xlsx
Aba "avarias" não encontrada no arquivo BRK - PORTO FERREIRA - 09-2024 (1).xlsx.
Aba carregada: Sublocação do arquivo BRK - PORTO FERREIRA - 09-2024 (1).xlsx
Aba "AVARIAS" não encontrada no arquivo BRK - RIO CLARO - 09-2024.xlsx.
Aba "Avarias" não encontrada no arquivo BRK - RIO CLARO - 09-2024.xlsx.
Aba "avarias" não encontrada no arquivo BRK - RIO CLARO - 09-2024.xlsx.
Aba "Sublocação" não encontrada no arquivo BRK - RIO CLARO - 09-2024.xlsx.
Aba "AVA

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
import pandas as pd
from google.colab import files
from openpyxl import load_workbook
from openpyxl.styles import PatternFill, Alignment, Font
from datetime import datetime, timedelta

def format_value(value):
    """Formata o valor numérico em formato monetário brasileiro."""
    if isinstance(value, (int, float)):
        return f"R$ {value:,.2f}".replace(',', 'v').replace('.', ',').replace('v', '.')
    else:
        value = value.replace("'", ".")
        return f"R$ {float(value):,.2f}".replace(',', 'v').replace('.', ',').replace('v', '.')

uploaded = files.upload()

sheet_names = ['AVARIAS', 'Avarias', 'avarias', 'Sublocação']
columns_to_extract_avarias = ['Tipo de Despesa', 'Valor do Reembolso', 'Nome do produto', 'Placa do dispositivo']
columns_to_extract_sublocacao = ['Placa Sublocado', 'TOTAL REEMBOLSAVEL II', 'Motivo do Chamado', 'Descrição']

combined_data = pd.DataFrame()

for filename in uploaded.keys():
    if filename.endswith(('.xlsx', '.xls')):
        for sheet in sheet_names:
            try:
                df = pd.read_excel(filename, sheet_name=sheet)
                print(f'Aba carregada: {sheet} do arquivo {filename}')

                if 'Valor do Reembolso' in df.columns and 'Tipo de Despesa' in df.columns:

                    last_item_value = df['Tipo de Despesa'].iloc[-1] if not df.empty else None

                    if pd.isna(last_item_value) or last_item_value == '':

                        df = df.iloc[:-1]

                if sheet in ['AVARIAS', 'Avarias', 'avarias']:
                    extracted_data = df[columns_to_extract_avarias].copy()

                    if 'Placa' in df.columns:
                        extracted_data.rename(columns={'Placa': 'Veículo'}, inplace=True)
                    elif 'Placa do dispositivo' in df.columns:
                        extracted_data.rename(columns={'Placa do dispositivo': 'Veículo'}, inplace=True)

                    extracted_data.rename(columns={'Tipo de Despesa': 'Item', 'Valor do Reembolso': 'Valor total', 'Nome do produto': 'Observação'}, inplace=True)
                    extracted_data['Item'] = extracted_data['Item'].replace('Avaria', 'Manutenção/Avaria')
                    extracted_data['Data/hora'] = ''
                    extracted_data['Colaborador'] = ''
                    extracted_data['Fornecedor'] = 'UNIDAS'
                    extracted_data['n°documento'] = ''
                    extracted_data['Quantidade'] = '1'
                    extracted_data['Valor Unitário'] = ''
                    extracted_data['Valor'] = ''
                    extracted_data['Hodometro'] = ''
                    extracted_data['Garantia Dias'] = ''
                    extracted_data['Garantia ultilizacao'] = ''
                    extracted_data['Desconto Reais'] = ''
                    extracted_data['Desconto %'] = ''
                    extracted_data['Valor total'] = extracted_data['Valor total'].apply(format_value)
                    combined_data = pd.concat([combined_data, extracted_data], ignore_index=True)

                elif sheet == 'Sublocação':
                    sublocacao_extracted = df[columns_to_extract_sublocacao].copy()
                    sublocacao_extracted.rename(columns={'Placa Sublocado': 'Veículo', 'TOTAL REEMBOLSAVEL II': 'Valor total', 'Motivo do Chamado': 'Item', 'Descrição': 'Observação'}, inplace=True)
                    sublocacao_extracted['Fornecedor'] = 'UNIDAS'
                    sublocacao_extracted['Item'] = sublocacao_extracted['Item'].replace('Avaria', 'Manutenção/Avaria')
                    sublocacao_extracted['Quantidade'] = '1'
                    sublocacao_extracted['Valor total'] = sublocacao_extracted['Valor total'].apply(format_value)
                    combined_data = pd.concat([combined_data, sublocacao_extracted], ignore_index=True)

            except ValueError:
                print(f'Aba "{sheet}" não encontrada no arquivo {filename}.')

date_input = input("Digite a data e hora (dd/mm/aaaa HH:MM): ")
initial_datetime = datetime.strptime(date_input, "%d/%m/%Y %H:%M")

combined_data['Data/hora'] = [
    (initial_datetime + timedelta(minutes=i)).strftime("%d/%m/%Y %H:%M:%S")
    for i in range(len(combined_data))
]

new_order = ['Data/hora', 'Colaborador', 'Fornecedor', 'n°documento', 'Veículo', 'Item', 'Quantidade', 'Valor Unitário', 'Valor', 'Valor total', 'Hodometro', 'Garantia Dias', 'Garantia ultilizacao', 'Desconto Reais', 'Desconto %', 'Observação']
combined_data = combined_data.reindex(columns=new_order)

new_filename = 'dados_combinados.xlsx'
combined_data.to_excel(new_filename, index=False)
print(f'Dados combinados salvos em: {new_filename}')


workbook = load_workbook(new_filename)
sheet = workbook.active

columns_to_paint = ['Data/hora', 'Fornecedor', 'Veículo', 'Item', 'Quantidade', 'Valor total', 'Observação']

for col in range(1, sheet.max_column + 1):
    cell = sheet.cell(row=1, column=col)
    cell.alignment = Alignment(horizontal='left')
    cell.font = Font(bold=False)

    if combined_data.columns[col - 1] in columns_to_paint:
        cell.fill = PatternFill(start_color='FFFF00', end_color='FFFF00', fill_type='solid')

for column in sheet.columns:
    max_length = max([len(str(cell.value)) for cell in column if cell.value]) + 2
    column_letter = column[0].column_letter
    sheet.column_dimensions[column_letter].width = max_length

workbook.save(new_filename)

files.download(new_filename)


Saving BRK AMBIENTAL - LIMEIRA S.A. - Set.24.xlsx to BRK AMBIENTAL - LIMEIRA S.A. - Set.24.xlsx
Saving BRK AMBIENTAL - PORTO FERREIRA S.A. - Set.24.xlsx to BRK AMBIENTAL - PORTO FERREIRA S.A. - Set.24.xlsx
Saving BRK AMBIENTAL - RIO CLARO S.A. - Set.24.xlsx to BRK AMBIENTAL - RIO CLARO S.A. - Set.24.xlsx
Saving BRK AMBIENTAL - SANTA GERTRUDES S.A - Set.24.xlsx to BRK AMBIENTAL - SANTA GERTRUDES S.A - Set.24.xlsx
Saving BRK AMBIENTAL - SUMARE S.A - Set.24.xlsx to BRK AMBIENTAL - SUMARE S.A - Set.24.xlsx
Saving Medicao_Unidas (CORRETA).xlsx to Medicao_Unidas (CORRETA).xlsx
Aba "AVARIAS" não encontrada no arquivo BRK AMBIENTAL - LIMEIRA S.A. - Set.24.xlsx.
Aba carregada: Avarias do arquivo BRK AMBIENTAL - LIMEIRA S.A. - Set.24.xlsx
Aba "avarias" não encontrada no arquivo BRK AMBIENTAL - LIMEIRA S.A. - Set.24.xlsx.
Aba "Sublocação" não encontrada no arquivo BRK AMBIENTAL - LIMEIRA S.A. - Set.24.xlsx.
Aba "AVARIAS" não encontrada no arquivo BRK AMBIENTAL - PORTO FERREIRA S.A. - Set.24.xlsx.

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>